# Lecture 4. CNN Build (VGG)

## 1. 환경 설정

In [40]:
# Colab에서는 pytorch 설치 필요
# !pip install torch torchvision

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import skimage
from skimage import transform as skit

print(torch.__version__)

2.0.0+cu117


## 2. Data 불러오기 및 전처리

In [64]:
data_path = '/home/student/Datasets/jhjeong/Test/' # 자신의 환경에 맞게 설정!

mnist_train = dset.MNIST(root=data_path, train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST(root=data_path, train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

In [65]:
train_x, test_x = [], []
for i in range(len(mnist_train)):
    train_x.append(skit.resize(mnist_train[i][0].numpy(), (1,64,64)))
print(np.array(train_x).shape)
for i in range(len(mnist_test)):
    test_x.append(skit.resize(mnist_test[i][0].numpy(), (1,64,64)))
print(np.array(test_x).shape)

(60000, 1, 64, 64)
(10000, 1, 64, 64)


In [61]:
print(len(mnist_train), len(mnist_test))
print(mnist_train[0][0].size()) # 0번째 프레임의 이미지 크기 출력
print(mnist_train[0][1]) # 0번째 프레임의 레이블(정답) 출력
plt.figure(figsize=(12,12))
for i in range(3*3): # 9개의 데이터 추가 확인
    plt.subplot(3,3,i+1)
    plt.imshow(np.moveaxis(mnist_train[i][0].numpy(), 0, -1)) # np.moveaxis()는 channel 위치를 0 -> -1로 옮겨준다.
    plt.title(f'label = {mnist_train[i][1]}')

60000 10000


TypeError: 'tuple' object is not callable

In [10]:
batch_size = 200

train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True,num_workers=2,drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False,num_workers=2,drop_last=True)
print(len(train_loader), len(test_loader))

300 50


## 3. 모델 만들기

In [ ]:
# 컨볼루션 연산이 2번 연속하는 경우
# 컨볼루션-활성화함수-컨볼루션-활성화함수-풀링
def conv_2_block(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

  
# 컨볼루션 연산이 3번 연속하는 경우
# 컨볼루션-활성화함수-컨볼루션-활성화함수-컨볼루션-활성화함수-풀링
def conv_3_block(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

In [12]:
# 위에서 정의한 블록들을 이용해 VGG 네트워크를 만들어보겠습니다.
# 필터의 개수가 2의 n승의 값을 가지기 때문에 base_dim이란 변수를 추가해서 단순화 했습니다.
# 현재 dog, cat 두 가지 클래스를 구분하려고 하기 때문에 num_classes=2로 설정했습니다.

class VGG(nn.Module):
    def __init__(self, base_dim, num_classes=2):
        super(VGG, self).__init__()
        self.feature = nn.Sequential(
            self.conv_block(1,base_dim),
            self.conv_block(base_dim,2*base_dim),
            self.conv_block(2*base_dim,4*base_dim),
            self.conv_block(4*base_dim,8*base_dim),
            self.conv_block(8*base_dim,8*base_dim),         
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(8*base_dim * 2 * 2, 100),
            nn.ReLU(True),                                                      # True 는 inplace 연산을 하겠다는 의미를 가집니다. inplace 연산은 결과값을 새로운 변수에 값을 저장하는 대신 기존의 데이터를 대체하는것을 의미합니다.
            #nn.Dropout(),
            nn.Linear(100, 20),
            nn.ReLU(True),
            #nn.Dropout(),
            nn.Linear(20, num_classes),
        )

    def forward(self, x):
        x = self.feature(x)
        x = x.view(x.size(0), -1)                                               # x.size(0)를 batch size로 바꿔도 같은 값입니다.
        x = self.fc_layer(x)
        return x
    
    def conv_block(self, in_dim,out_dim, num):
        layer_list = []
        for i in range(num):
            layer_list.append(nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1))
            layer_list.append(nn.ReLU())
        layer_list.append(nn.MaxPool2d(2,2))
        return nn.Sequential(layer_list)
    
